## Day 22: Sporifica Virus

http://adventofcode.com/2017/day/22

### Post-mortem

Let's amend this to use mutable data structures.

In [10]:
from collections import namedtuple, deque


State = namedtuple('State', 'weakened infected flagged virus_p virus_d total_infections')


def initialise(data):
    def up(p): return (p[0]-1, p[1])
    def right(p): return (p[0], p[1]+1)
    def down(p): return (p[0]+1, p[1])
    def left(p): return (p[0], p[1]-1)

    weakened = set()
    infected = set({(r, c) 
                    for r, row in enumerate(data) 
                    for c, ch in enumerate(row) 
                    if ch == '#'})
    flagged = set()
    
    virus_position = (len(data) // 2, len(data[0]) // 2)
    virus_direction = deque([up, right, down, left])
    
    return State(weakened, infected, flagged, virus_position, virus_direction, 0)


def move(state):
    weakened, infected, flagged, v_p, v_d, infections = state
    
    if v_p in infected:
        # turn right
        v_d.rotate(-1)
        # flag
        infected.remove(v_p)
        flagged.add(v_p)
    elif v_p in weakened:
        # infect
        weakened.remove(v_p)
        infected.add(v_p)
        infections += 1
    elif v_p in flagged:
        # turn around
        v_d.rotate(2)
        # clean
        flagged.remove(v_p)
    else: # clean
        # turn left
        v_d.rotate(1)
        # weaken
        weakened.add(v_p)
        
    v_p = v_d[0](v_p)
    
    return State(weakened, infected, flagged, v_p, v_d, infections)


def run_n(data, n):
    s = initialise(data)
    
    for _ in range(n):
        s = move(s)
        
    return s

In [11]:
test_data = '''..#
#..
...'''.splitlines()

run_n(test_data, 100)

State(weakened={(-2, 0), (2, -4), (2, 1), (0, -2), (4, -1), (4, 0), (-1, 1), (2, 2), (0, 4), (-2, -1), (2, -2), (-1, 2), (2, 3), (1, -4), (1, 0), (-1, 3), (-1, 4)}, infected={(0, 0), (-1, 0), (3, 0), (0, 2), (2, -1), (3, -1), (-1, -1), (0, -1), (1, 1)}, flagged={(1, 2), (0, 1), (2, 0), (1, 3), (2, -3), (1, -2), (1, -3)}, virus_p=(2, 0), virus_d=deque([<function initialise.<locals>.up at 0x7f151090b2f0>, <function initialise.<locals>.right at 0x7f151090b400>, <function initialise.<locals>.down at 0x7f151090b378>, <function initialise.<locals>.left at 0x7f151090b0d0>]), total_infections=26)

In [12]:
%time test_final_state = run_n(test_data, 10000000)

CPU times: user 15.6 s, sys: 2.69 ms, total: 15.6 s
Wall time: 15.6 s


That's compared to five and a half minutes with pyrsistent's persistent immutable structures.

With pypy the immutable version ran in just over a minute.

```bash
$ time pypy3 sporifica_virus.py 
2512059

real	0m3.538s
user	0m3.526s
sys	0m0.012s
```